In [1]:
import os
GPU_id = 7
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_id)

In [2]:
import cuml
import cudf
import cupy

import logging
import anndata as ad
import numpy as np

from scipy.sparse import csc_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.gaussian_process.kernels import RBF
from sklearn.kernel_ridge import KernelRidge
import os
from glob import glob
from tqdm import tqdm

import gc

In [3]:
path = '/raid/data/ml/nips/datasets/predict_modality'
os.listdir(path)

['openproblems_bmmc_cite_phase2_mod2',
 'openproblems_bmmc_multiome_phase2_mod2',
 'openproblems_bmmc_cite_phase2_rna',
 'openproblems_bmmc_multiome_phase2_rna']

In [4]:
tasks = ["GEX2ADT", "ADT2GEX", "GEX2ATAC", "ATAC2GEX"]
task2name = {
    "ADT2GEX":f"openproblems_bmmc_cite_phase2_mod2",
    "GEX2ADT":f"openproblems_bmmc_cite_phase2_rna",
    "ATAC2GEX":f"openproblems_bmmc_multiome_phase2_mod2",
    "GEX2ATAC":f"openproblems_bmmc_multiome_phase2_rna"
}

In [5]:
task = "GEX2ADT"
name = task2name[task]
tag = "train_mod1"
glob(f'{path}/{name}/*{tag}*.h5ad')[0]

'/raid/data/ml/nips/datasets/predict_modality/openproblems_bmmc_cite_phase2_rna/openproblems_bmmc_cite_phase2_rna.censor_dataset.output_train_mod1.h5ad'

In [6]:
tags = ['train_mod1', 'train_mod2', 'test_mod1', 'test_mod2']
par = {f'input_{tag}': glob(f'{path}/{name}/*{tag}*.h5ad')[0] for tag in tags}
par

{'input_train_mod1': '/raid/data/ml/nips/datasets/predict_modality/openproblems_bmmc_cite_phase2_rna/openproblems_bmmc_cite_phase2_rna.censor_dataset.output_train_mod1.h5ad',
 'input_train_mod2': '/raid/data/ml/nips/datasets/predict_modality/openproblems_bmmc_cite_phase2_rna/openproblems_bmmc_cite_phase2_rna.censor_dataset.output_train_mod2.h5ad',
 'input_test_mod1': '/raid/data/ml/nips/datasets/predict_modality/openproblems_bmmc_cite_phase2_rna/openproblems_bmmc_cite_phase2_rna.censor_dataset.output_test_mod1.h5ad',
 'input_test_mod2': '/raid/data/ml/nips/datasets/predict_modality/openproblems_bmmc_cite_phase2_rna/openproblems_bmmc_cite_phase2_rna.censor_dataset.output_test_mod2.h5ad'}

In [7]:
def print_shape(*x):
    for i in x:
        print(i.shape, end=' ')
    print()

In [8]:
%%time
input_train_mod1 = ad.read_h5ad(par['input_train_mod1'])
input_train_mod2 = ad.read_h5ad(par['input_train_mod2'])
input_test_mod1 = ad.read_h5ad(par['input_test_mod1'])
input_test_mod2 = ad.read_h5ad(par['input_test_mod2'])

print_shape(input_train_mod1, input_train_mod2, input_test_mod1, input_test_mod2)

(66175, 13953) (66175, 134) (1000, 13953) (1000, 134) 
CPU times: user 6.28 s, sys: 845 ms, total: 7.13 s
Wall time: 7.12 s


In [ ]:
%%time

pred_dimx = input_test_mod1.shape[0]
pred_dimy = input_train_mod2.shape[1]

feature_obs = input_train_mod1.obs
gs_obs = input_train_mod2.obs

batches = input_train_mod1.obs.batch.unique().tolist()
batch_len = len(batches)

obs = input_test_mod1.obs
var = input_train_mod2.var
dataset_id = input_train_mod1.uns['dataset_id']

input_train = ad.concat(
    {"train": input_train_mod1, "test": input_test_mod1},
    axis=0,
    join="outer",
    label="group",
    fill_value=0,
    index_unique="-"
)

In [ ]:
print('Determine parameters by the modalities')
mod1_type = input_train_mod1.var.feature_types[0]
mod1_type = mod1_type.upper()
mod2_type = input_train_mod2.var.feature_types[0]
mod2_type = mod2_type.upper()
mod1_type, mod2_type

In [ ]:
n_comp_dict = {
        ("GEX", "ADT"): (300, 70, 10, 0.2),
        ("ADT", "GEX"): (None, 50, 10, 0.2),
        ("GEX", "ATAC"): (1000, 50, 10, 0.1),
        ("ATAC", "GEX"): (100, 70, 10, 0.1)
        }
print(f"{mod1_type}, {mod2_type}")
n_mod1, n_mod2, scale, alpha = n_comp_dict[(mod1_type, mod2_type)]
print(f"{n_mod1}, {n_mod2}, {scale}, {alpha}")

# Do PCA on the input data
print('Models using the Truncated SVD to reduce the dimension')

In [ ]:
%%time

X = cupy.asarray(input_train.X.toarray(), order='F')
y = cupy.asarray(input_train_mod2.X.toarray(), order='F')

In [ ]:
%%time

def get_mask(ds, val):
    r = ds == val
    return r.values

if n_mod1 is not None and n_mod1 < input_train.shape[1]:
    embedder_mod1 = cuml.decomposition.TruncatedSVD(n_components=n_mod1)
    mod1_pca = embedder_mod1.fit_transform(X).astype(np.float32)
    train_matrix = mod1_pca[get_mask(input_train.obs['group'],'train')]
    test_matrix = mod1_pca[get_mask(input_train.obs['group'],'test')]
else:
    train_matrix = input_train_mod1.to_df().values.astype(np.float32)
    test_matrix = input_test_mod1.to_df().values.astype(np.float32)

if n_mod2 is not None and n_mod2 < input_train_mod2.shape[1]:
    embedder_mod2 = cuml.decomposition.TruncatedSVD(n_components=n_mod2)
    train_gs = embedder_mod2.fit_transform(y).astype(np.float32)
else:
    train_gs = input_train_mod2.to_df().values.astype(np.float32)

del input_train
del input_train_mod1
del input_train_mod2
del input_test_mod1
del X, y
gc.collect()

In [ ]:
%%time

print('Running normalization ...')
train_sd = np.std(train_matrix, axis=1).reshape(-1, 1)
train_sd[train_sd == 0] = 1
train_norm = (train_matrix - np.mean(train_matrix, axis=1).reshape(-1, 1)) / train_sd
train_norm = train_norm.astype(np.float32)
del train_matrix

test_sd = np.std(test_matrix, axis=1).reshape(-1, 1)
test_sd[test_sd == 0] = 1
test_norm = (test_matrix - np.mean(test_matrix, axis=1).reshape(-1, 1)) / test_sd
test_norm = test_norm.astype(np.float32)
del test_matrix

In [ ]:
cuml.metrics.PAIRWISE_KERNEL_FUNCTIONS

In [ ]:
import numpy as np
import warnings
from cupy import linalg
import cupy as cp
from cupyx import lapack, geterr, seterr
from cuml.common.array_descriptor import CumlArrayDescriptor
from cuml.common.base import Base
from cuml.common.mixins import RegressorMixin
from cuml.common.doc_utils import generate_docstring
from cuml.common import input_to_cuml_array

from cuml.metrics import pairwise_kernels


# cholesky solve with fallback to least squares for singular problems
def _safe_solve(K, y):
    try:
        # we need to set the error mode of cupy to raise
        # otherwise we silently get an array of NaNs
        err_mode = geterr()["linalg"]
        seterr(linalg="raise")
        dual_coef = lapack.posv(K, y)
        seterr(linalg=err_mode)
    except np.linalg.LinAlgError:
        warnings.warn(
            "Singular matrix in solving dual problem. Using "
            "least-squares solution instead."
        )
        dual_coef = linalg.lstsq(K, y, rcond=None)[0]
    return dual_coef


def _solve_cholesky_kernel(K, y, alpha, sample_weight=None):
    # dual_coef = inv(X X^t + alpha*Id) y
    n_samples = K.shape[0]
    n_targets = y.shape[1]

    K = cp.array(K, dtype=np.float64)

    alpha = cp.atleast_1d(alpha)
    one_alpha = alpha.size == 1
    has_sw = sample_weight is not None

    if has_sw:
        # Unlike other solvers, we need to support sample_weight directly
        # because K might be a pre-computed kernel.
        sw = cp.sqrt(cp.atleast_1d(sample_weight))
        y = y * sw[:, cp.newaxis]
        K *= cp.outer(sw, sw)

    if one_alpha:
        # Only one penalty, we can solve multi-target problems in one time.
        K.flat[:: n_samples + 1] += alpha[0]

        dual_coef = _safe_solve(K, y)

        if has_sw:
            dual_coef *= sw[:, cp.newaxis]

        return dual_coef
    else:
        # One penalty per target. We need to solve each target separately.
        dual_coefs = cp.empty([n_targets, n_samples], K.dtype)

        for dual_coef, target, current_alpha in zip(dual_coefs, y.T, alpha):
            K.flat[:: n_samples + 1] += current_alpha

            dual_coef[:] = _safe_solve(K, target).ravel()

            K.flat[:: n_samples + 1] -= current_alpha

        if has_sw:
            dual_coefs *= sw[cp.newaxis, :]

        return dual_coefs.T
class KernelRidgeMultiOutput(cuml.kernel_ridge.KernelRidge):

    def fit(self, X, y, sample_weight=None,
            convert_dtype=True) -> "KernelRidge":

        ravel = False
        if len(y.shape) == 1:
            y = y.reshape(-1, 1)
            ravel = True

        X_m, n_rows, self.n_cols, self.dtype = input_to_cuml_array(
            X, check_dtype=[np.float32, np.float64]
        )

        y_m, _, _, _ = input_to_cuml_array(
            y,
            check_dtype=self.dtype,
            convert_to_dtype=(self.dtype if convert_dtype else None),
            check_rows=n_rows,
        )

        if self.n_cols < 1:
            msg = "X matrix must have at least a column"
            raise TypeError(msg)

        K = self._get_kernel(X_m)
        
        self.dual_coef_ = []
        y_m = y_m.to_output('cupy')
        for i in range(y_m.shape[1]):
            coef = _solve_cholesky_kernel(
                K, y_m[:,i], self.alpha, sample_weight
            )
            if ravel:
                coef = coef.ravel()
            self.dual_coef_.append(coef)
        self.X_fit_ = X_m
        return self

    def predict(self, X):
        """
        Predict using the kernel ridge model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Samples. If kernel == "precomputed" this is instead a
            precomputed kernel matrix, shape = [n_samples,
            n_samples_fitted], where n_samples_fitted is the number of
            samples used in the fitting for this estimator.

        Returns
        -------
        C : array of shape (n_samples,) or (n_samples, n_targets)
            Returns predicted values.
        """
        X_m, _, _, _ = input_to_cuml_array(
            X, check_dtype=[np.float32, np.float64])

        K = self._get_kernel(X_m, self.X_fit_)
        K = cp.asarray(K)
        yps = []
        for coef in self.dual_coef_:
            yps.append(cp.dot(K, cp.asarray(coef)))
        return cp.array(yps).T

In [ ]:
%%time

print('Running KRR model ...')
y_pred = cupy.zeros((pred_dimx, pred_dimy), dtype=np.float32)
cupy.random.seed(1000)
emb = embedder_mod2.components_.to_output('cupy')
for _ in tqdm(range(5)):
    np.random.shuffle(batches)
    for batch in [batches[:batch_len//2], batches[batch_len//2:]]:
        # for passing the test
        if not batch:
            batch = [batches[0]]

        logging.info(batch)
        
        print('Fitting KRR ... ')
        x = train_norm[feature_obs.batch.isin(batch).values]
        y = train_gs[gs_obs.batch.isin(batch).values]
        krr = cuml.kernel_ridge.KernelRidge(alpha=alpha, kernel='rbf')
        krr.fit(x, y)
        gc.collect()
        yp = krr.predict(test_norm)
        yp = yp @ emb
        y_pred += yp
        gc.collect()

In [ ]:
y_pred = y_pred.get()
np.clip(y_pred, a_min=0, a_max=None, out=y_pred)
if mod2_type == "ATAC":
    np.clip(y_pred, a_min=0, a_max=1, out=y_pred)

y_pred /= 10

# Store as sparse matrix to be efficient. Note that this might require
# different classifiers/embedders before-hand. Not every class is able
# to support such data structures.
y_pred = csc_matrix(y_pred)

logging.info("Generate anndata object ...")
adata = ad.AnnData(
    X=y_pred,
    obs=obs,
    var=var,
    uns={
        'dataset_id': dataset_id,
        'method_id': 'gpu',
    },
)

logging.info('Storing annotated data...')
adata.write_h5ad('gpu.h5ad', compression = "gzip")

In [19]:
yx = y_pred.toarray()

In [20]:
yx

array([[0.16873707, 0.23536715, 0.61948097, ..., 0.39800423, 0.42028886,
        0.52114236],
       [0.04233083, 0.28994787, 0.65432906, ..., 0.21106711, 0.36479193,
        0.24584863],
       [0.02861868, 0.2421951 , 0.6622361 , ..., 0.1731327 , 0.39247906,
        0.16301864],
       ...,
       [0.0712819 , 0.22787991, 0.53754294, ..., 0.28534326, 0.36496112,
        0.23056659],
       [0.0727509 , 0.18692048, 0.5490494 , ..., 0.15434405, 0.3627604 ,
        0.2049741 ],
       [0.0495175 , 0.3464053 , 0.6344657 , ..., 0.22001934, 0.38574556,
        0.31911534]], dtype=float32)